# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [4]:
import numpy as np

### ESOINN node class
##### Fields
* feature_vector – weights
* accamulate_signals – number of signals
* total_points – points $\neq$ number of signals
* density – mean accumulated signals
* subclass_id – mark for subclass

In [59]:
class ESOINN_Node:
    def __init__(self, feature_vector=()):
        self.feature_vector = np.array(feature_vector, dtype=float)  
        self.accamulate_signals = 0
        self.total_points = 0
        self.density = 0
        self.subclass_id = -1
    
    def update_accamulate_signals(self, n=1):
        self.accamulate_signals += 1

### ESOINN Neural Network class
To start lerning use: `fit()` method, for clasterization use `predict()`.

##### Params:
To create new `EnhancedSelfOrganizingIncrementalNN` object – initialize it with first two nodes (`init_nodes`) randomly.

##### Hiperparams:
* `C1`, `C2` – coefficents for noise deletion.
* `learning_step` – number of iterations before remove old ages and find classes ($\lambda$ in literature).
* `max_age` – for edges.
* `forget` – specify which N is used in density calculation.
* `metrics` – lambda(x, y, axis)
* `radius_cut_off` – degree of neighbors' nodes.
* `learning_rate_winner` 
* `learning_rate_winner_neighbor`

##### Fields:
* `ids` – last given id for nodes (should be unique).

In [21]:
class EnhancedSelfOrganizingIncrementalNN:
    def __init__(self, init_nodes, C1=0.001, C2=1, learning_step=200, max_age=50, 
                 metrics=lambda x,y,axis=0: np.sqrt(np.sum(np.square(np.array(x) - np.array(y)), axis=axis)),
                 forget=False, radius_cut_off=1, learning_rate_winner=lambda t: 1/t, 
                 learning_rate_winner_neighbor=lambda t: 1/(100*t)):
        self.C1 = C1
        self.C2 = C2
        self.learning_step = learning_step
        self.max_age = max_age
        self.signals_amount = 2
        self.metrics = metrics
        self.forget = forget
        self.unique_id = 2
        self.rc = radius_cut_off
        self.learning_rate_winner = learning_rate_winner
        self.learning_rate_winner_neighbor = learning_rate_winner_neighbor
        
        self.nodes = {i: ESOINN_Node(init_nodes[i]) for i in (0, 1)}
        self.neighbors = {}  # keys = id, values = sets of neighbors' ids
        self.edges = {}  # keys = tuples(2), where t[0] < t[1], value = age/None
    
    def fit(self, input_signal):
        self.signals_amount += 1
        
        winners_ids, distances = self.find_winners(input_signal)
        thresholds = [self.calc_threshold(input_signal, winners_ids[i]) for i in (0, 1)]
        if distances[0] > thresholds[0] or distances[1] > thresholds[1]:
            self.create_node(input_signal)
            return
        
        self.update_edges_age(winners_ids[0])
        self.build_connection(winners_ids)
        
#         @fixme обновление количества побед нейрона - не знаю куда поставить, но точно до подсчета плотности
        self.nodes[winners_ids[0]].update_accamulate_signals()
        
        self.update_density(winners_ids[0])
        
#         self.nodes[winners_ids[0]].update_accamulate_signals()
        
#         self.update_feature_vector(winners_ids[0])
        
#         self.remove_old_ages()
        
#         if self.signals_amount % self.learning_step == 0:
#             self.update_topology()
    
    def find_winners(self, input_signal):
        # @fixme: inf coef and separate variables for each winner
        winner1 = float('inf')
        winner1_id = -1
        winner2 = float('inf')
        winner2_id = -1
        for node_id in self.nodes:
            dist = self.metrics(input_signal, self.nodes[node_id].feature_vector)
            if dist <= winner1:
                winner1, winner2 = dist, winner1
                winner2_id = winner1_id
                winner1_id = node_id
            elif dist < winner2:
                winner2 = dist
                winner2_id = node_id
        return [winner1_id, winner2_id], [winner1, winner2]
    
    def find_neighbors(start_node_id, depth=1):
        visited = {start_node_id}
        queue = list(self.neighbors.get(start, set()) - visited)
        while depth:
            depth -= 1
            for vertex in queue.copy():  # @fixme: do not use copy!
                visited.add(vertex)
                queue.extend([node for node in self.neighbors[vertex] - visited if node not in visited])
        return visited - {start_node_id}    
    
    def calc_threshold(self, input_signal, winner_id):
        neighbors = self.neighbors.get(winner_id, None)
        if neighbors:
            return np.max([
                self.metrics(self.nodes[winner_id].feature_vector, self.nodes[neighbor_id].feature_vector) 
                for neighbor_id in find_neighbors(winner_id, depth=self.rc)
            ])
        else:
            return self.find_winners(self.nodes[winner_id].feature_vector)[1][1]  # 'cause first winner is always current node
    
    def create_node(self, input_signal):  
        self.nodes[self.unique_id] = ESOINN_Node(input_signal)
        self.unique_id += 1  # to provide unique ids for each neuron
    
    def update_edges_age(self, node_id, step=1):
        for neighbor_id in self.neighbors.get(node_id, []):
            pair_id = min(node_id, neighbor_id), max(node_id, neighbor_id)
            self.edges[pair_id] += 1
                
    # algorithm 3.2
    def build_connection(self, winners_ids):
        winner1_class = self.nodes[winners_ids[0]].subclass_id
        winner2_class = self.nodes[winners_ids[1]].subclass_id
        # case 1-2
        if winner1_class == -1 or winner2_class == -1 or winner1_class == winner2_class:
            self.create_edge(winners_ids)
        # case 3
        pass
                
    def create_edge(self, nodes_ids):
        for node_id in nodes_ids:
            if node_id not in self.neighbors:
                self.neighbors[node_id] = set()
            # @CHECKME: for multiply neighbors creation
            for insert_id in nodes_ids:
                if insert_id != node_id:
                    self.neighbors[node_id] |= {insert_id}
                    nodes_pair = (min(node_id, insert_id), max(node_id, insert_id))
                    self.edges[nodes_pair] = 0
    
    def remove_edge(self, nodes_ids):
        pass  # @todo: edge removal
        
    # @fixme работает вроде правильно, но может можно еще изящнее сделать? :D
    def update_density(self, winner_id):
        
#       подсчет total_points (Тут очень не здоровое но рабочее дерьмо :D )
        self.nodes[winner_id].total_points += 1 / (1 + np.sum([self.metrics(self.nodes[winner_id].feature_vector, 
            self.nodes[neighbor_id].feature_vector) for neighbor_id in self.neighbors[winner_id]]
            )/len(self.neighbors[winner_id]))**2

#       подсчет плотности
        self.nodes[winner_id].density = self.nodes[winner_id].total_points/self.nodes[winner_id].accamulate_signals

    def update_feature_vector(self, winner_id, input_signal):
        
#       смещение вектора признаков нейрона победителя
        self.nodes[winner_id].feature_vector += self.learning_rate_winner(self.nodes[winner_id].accamulate_signals)*(
            self.metrics(self.nodes[winner_id].feature_vector, input_signal))

#       смещение вектора признаков смежных нейрону победителю
        for neighbor_id in self.neighbors[winner_id]:
            self.nodes[neighbor_id].feature_vector += self.learning_rate_winner_neighbor(
                self.nodes[winner_id].accamulate_signals)*self.metrics(self.nodes[neighbor_id].feature_vector, 
                input_signal)
    
    def remove_old_ages(self):
        
#       удаление ребра и удаление соседей у нейронов, если превышен порог max_age
        [(self.neighbors[edge[0]].remove(edge[1]), self.neighbors[edge[1]].remove(edge[0]), self.edges.pop(edge)) 
         for edge in self.edges.copy() if self.edges[edge] > self.max_age]
        
#       удаление пустых set-ов у нейронов в neighbor
        [self.neighbors.pop(node_id) for node_id in self.nodes if self.neighbors[node_id] == set()]

#     def predict(self, input_signal):
#         pass

#     def update(self):
#         pass
    
    def current_state(self):
        return {
            'signals_amount': self.signals_amount,
            'C1': self.C1,
            'C2': self.C2,
            'lambda': self.learning_step,
            'forget': self.forget,
            'max_age': self.max_age,
            'metrics': self.metrics,
            'learning_rate_winner': self.learning_rate_winner,
            'learning_rate_winner_neighbor': self.learning_rate_winner_neighbor,
            'nodes': self.nodes,  # think about it
            'neighbors': self.neighbors,
            'edges': self.edges
        }

### tests
@TODO: Should be automated.

In [25]:
input_signal = np.array([2, 2])  # [2,19] for node&edge creation
nn = EnhancedSelfOrganizingIncrementalNN([[1, 2], [5, 2]])

nn.fit(input_signal)
# test for old edge removal
# nn.edges[(0,1)] = 51
# nn.remove_old_ages()

nn_info = nn.current_state()  # this is more correct

print("TEST nodes feature vector and density:")
for i in nn_info['nodes'].keys():
    print(f"node {i} : {nn_info['nodes'][i].feature_vector} | density {nn_info['nodes'][i].density}")

print("\nTEST neighbours for nodes:")
for i in range(len(nn_info['neighbors'])):
    print(f"neighbors for node {i} = {nn_info['neighbors'].get(i, None)}")

print("\nTEST edges between nodes:")
print(f"edge between winners (node 0, node 1) = {nn_info['edges'].get((0,1))}")
print(f"edge between winners (None) = {nn_info['edges'].get(())}")

TEST nodes feature vector and density:
node 0 : [1 2] | density 0.04
node 1 : [5 2] | density 0

TEST neighbours for nodes:
neighbors for node 0 = {1}
neighbors for node 1 = {0}

TEST edges between nodes:
edge between winners (node 0, node 1) = 0
edge between winners (None) = None
